In [ ]:
!pip install torch lerobot

In [ ]:
!pip install lerobot.data

## Training

In [ ]:
# Add parent directory to path to import train module
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from train import train

print("\nStarting training process...")
print("Note: We'll use a public dataset for training as our sample is too small")

# Create a temporary directory for training output
train_output_dir = Path('model_output')

print(f"Training output will be saved to: {train_output_dir}")

train(output_dir=str(train_output_dir), dataset_id="ISdept/piper_arm")

## Prepare dataset

In [2]:
# Import necessary modules
import sys
from pathlib import Path

# Add the src directory to the path so we can import prepare_dataset


from data_processing.prepare_dataset import process_session, create_tasks_parquet, create_episodes_parquet_index
import json
from data_processing.episode_data import EpisodeData, CameraData

# --- CONFIGURATION ---
DATASET_ROOT_DIR = Path("output")
SESSION_JSON_PATH = "/Users/eddyma/DEV/Github/lerobot-piper/src/data_processing/sample_data/metadata_20251030_181951.json"
REPO_ID = "ISDept/piper_arm" # Your desired Hugging Face repo ID
EPISODE_IDX = 0 # If you are processing multiple JSON files, increment this for each file
global_index_offset = 0
# ---------------------

cameras_list = [
    CameraData(video_path="/Users/eddyma/DEV/Github/lerobot-piper/src/data_processing/sample_data/robot_session_20251030_181951_front.mp4", camera="front_camera"),
    CameraData(video_path="/Users/eddyma/DEV/Github/lerobot-piper/src/data_processing/sample_data/robot_session_20251030_181951_rear.mp4", camera="rear_camera")
]

episode_data = EpisodeData(SESSION_JSON_PATH, EPISODE_IDX, fps=10, global_index_offset=global_index_offset, cameras=cameras_list)

process_session(episode_data, DATASET_ROOT_DIR)

create_tasks_parquet(DATASET_ROOT_DIR, 'pick_and_place')

create_episodes_parquet_index(DATASET_ROOT_DIR, EPISODE_IDX)


--- Generating videos ---

--- Generating meta ---
--- Creating meta/tasks.parquet ---
✅ Successfully created tasks.parquet at: output/meta/tasks.parquet

--- Creating meta/episodes/ index dataset ---
✅ Successfully created episodes index dataset at: output/meta/episodes

--- Generating data files ---


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 962.22ba/s]

✅ Successfully processed episode 0
--- Creating meta/tasks.parquet ---
✅ Successfully created tasks.parquet at: output/meta/tasks.parquet

--- Creating meta/episodes/ index dataset ---
✅ Successfully created episodes index dataset at: output/meta/episodes


In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import HfApi
import os


!hf upload \
  'ISDept/piper_arm' \
  /Users/eddyma/DEV/Github/lerobot-piper/src/output \
  --repo-type dataset

## Inference

In [10]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))

!python webcam_inference.py

Webcam LeRobot Inference Demo
Loading model...
Loading dataset metadata...
Fetching 4 files: 100%|█████████████████████████| 4/4 [00:00<00:00, 1198.63it/s]
Creating policy configuration...
Initializing policy...
Model weights not found at model_output/pytorch_model.bin
Could not load trained model. Using demo mode.

To use with a real trained model:
1. Train a model using train.py
2. Ensure the model is saved to ./model_output
3. Make sure lerobot is installed

DEMO MODE - Webcam Only
Webcam 0 initialized successfully
Webcam initialized successfully
Displaying webcam feed. Press 'q' to quit.
Processing frame...
Processing frame...
Processing frame...
Processing frame...
Processing frame...
Processing frame...
Processing frame...
^C
Traceback (most recent call last):
  File "/Users/eddyma/DEV/Github/lerobot-piper/src/webcam_inference.py", line 384, in <module>
    main()
  File "/Users/eddyma/DEV/Github/lerobot-piper/src/webcam_inference.py", line 365, in main
    inference_engine.clean